In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install nltk tensorflow-hub

    100% |################################| 1.4MB 8.5MB/s eta 0:00:01    27% |########                        | 389kB 3.2MB/s eta 0:00:01
    100% |################################| 61kB 7.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built nltk
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import preprocessor
import pandas as pd
import pickle
import os

In [3]:
def maybe_process(store_file="data.pkl", force=False):
    if force or not os.path.exists(store_file):
        data = preprocessor.process_dir("dump")
        with open(store_file, 'wb') as data_file:
            pickle.dump(data, data_file)
    else:
        with open(store_file, 'rb') as data_file:
            data = pickle.load(data_file)
    return data

In [4]:
data = maybe_process()

In [5]:
df = pd.DataFrame(data)
df = df.fillna("Unknown")

In [6]:
user_vocabulary = pd.concat([df["assignee"], df["reporter"]]).unique()

In [7]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

In [8]:
vocabulary_map = {"Unknown": 0}
count = 0
for el in user_vocabulary:
    count += 1
    vocabulary_map[el] = count

In [9]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

with open("vocabulary_map.pkl", 'wb') as vocabulary_file:
    pickle.dump(vocabulary_map, vocabulary_file)

Start tensorflow stuff here

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import shutil
import numpy as np

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

1.9.0


In [11]:
df.head()

,assignee,description,id,key,reporter,status,summary
0,yh02364,Broken links to account_xapi logs and conf on ...,825488,DEVOPS-8155,yh02364,Closed,Broken links to account_xapi logs and conf on ...
1,yh00677,https://c4d.devops.fds.com/#pk=8143 \r\n\r\nT...,683408,WDSDO-25052,p139rpd,Closed,Delete C4D Env
2,yc03k1m,Please grant access to Suvarna (m172665) tpo J...,753374,DEVOPS-2558,m910042,Closed,Suvarna Kommura requires QA access to Jenkins ...
3,yc14sv1,Parameters that used in deployment or redeploy...,299398,WDSDO-5800,yc14ay1,Closed,Request to provide parameters of Webapp enviro...
4,m825519,Create new certification dashboards for MCOM a...,638405,WDSDO-22797,m825519,Closed,Certification dashboard per brand


In [12]:
df.describe()

,assignee,description,id,key,reporter,status,summary
count,41397,41397,41397,41397,41397,41397,41397
unique,1121,40159,41397,41397,2151,15,39824
top,yc14tz1,TBD,796369,UPL-2861,yc14sv1,Closed,Discovery build failed
freq,3832,104,1,1,1002,36637,22


Now, split the data into two parts -- training and evaluation.

In [13]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

In [14]:
def add_more_features(df):
    # TODO: Add more features to the dataframe
    return df

In [15]:
# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = add_more_features(df),
#         y = df["assignee"].fillna('Unknown').apply(lambda x: vocabulary_map[x]),
        y = df["assignee"],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
      )

In [16]:
# Define your feature columns
def create_feature_cols():
    return [
#         tf.feature_column.categorical_column_with_vocabulary_list("reporter", 
#                                                                   vocabulary_list=user_vocabulary.tolist(), 
#                                                                   default_value=0)

        hub.text_embedding_column(
            key="description", 
            module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
            trainable=True,
        )
        
    # TODO: Define additional feature columns
    # Hint: Are there any features that would benefit from bucketizing?
    ]

In [17]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNClassifier(
        model_dir = output_dir,
        feature_columns = create_feature_cols(),
        hidden_units=[500, 100],
        n_classes=len(user_vocabulary.tolist()),
        label_vocabulary = user_vocabulary.tolist(),
    )
    
#     estimator = tf.estimator.LinearClassifier(
#         model_dir = output_dir, 
#         feature_columns = create_feature_cols(),
#         n_classes=len(user_vocabulary.tolist()),
#         label_vocabulary = user_vocabulary.tolist(),
#     )
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None),
                                        max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                      steps = None, 
                                      start_delay_secs = 1, # start evaluating after N seconds, 
                                      throttle_secs = 5)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [18]:
OUTDIR = './trained_model'

In [ ]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 2000)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/nnlm-en-dim128/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/nnlm-en-dim128/1'.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_master': '', '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3cfbd66860>, '_device_fn': None, '_session_config': None, '_global_id_in_cluster': 0, '_evaluation_master': '', '_task_id': 0, '_is_chief': True, '_service': None, '_num_ps_replicas': 0, '_model_dir': './trained_model', '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker'}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflo

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 906.75073, step = 5
INFO:tensorflow:Saving checkpoints for 5 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 906.75073.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:41:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-5
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-25-02:41:07
INFO:tensorflow:Saving dict for global step 5: accuracy = 0.027428709, average_lo

INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 701.14197, step = 10
INFO:tensorflow:Saving checkpoints for 10 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 701.14197.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:42:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished ev

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-14
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 14 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 638.3835, step = 15
INFO:tensorflow:Saving checkpoints for 15 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 638.3835.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:44:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mode

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-19
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 19 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 619.20746, step = 20
INFO:tensorflow:Saving checkpoints for 20 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 619.20746.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:45:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mo

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-24
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 24 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 639.7382, step = 25
INFO:tensorflow:Saving checkpoints for 25 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 639.7382.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:47:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mode

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-29
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 29 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 614.0807, step = 30
INFO:tensorflow:Saving checkpoints for 30 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 614.0807.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:49:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mode

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-34
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 34 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 586.21533, step = 35
INFO:tensorflow:Saving checkpoints for 35 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 586.21533.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:50:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mo

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-39
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 39 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 605.5737, step = 40
INFO:tensorflow:Saving checkpoints for 40 into ./trained_model/model.ckpt.
INFO:tensorflow:Loss for final step: 605.5737.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/description_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-25-02:52:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/mode